# Headers ISMRMRD

In [ ]:
from pathlib import Path
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
import math
from typing import List, Dict, Tuple, Union


# All patient IDs to consider for Uncertainty Quantification
pat_ids = [
    # '0003_ANON5046358',
    # '0004_ANON9616598',
    # '0005_ANON8290811',
    # '0006_ANON2379607',
    '0007_ANON1586301',
    # '0008_ANON8890538',
    # '0010_ANON7748752',
    # '0011_ANON1102778',
    # '0012_ANON4982869',
    # '0013_ANON7362087',
    # '0014_ANON3951049',
    # '0015_ANON9844606',
    # '0018_ANON9843837',
    # '0019_ANON7657657',
    # '0020_ANON1562419',
    # '0021_ANON4277586',
    # '0023_ANON6964611',
    # '0024_ANON7992094',
    # '0026_ANON3620419',
    # '0027_ANON9724912',
    # '0028_ANON3394777',
    # '0029_ANON7189994',
    # '0030_ANON3397001',
    # '0031_ANON9141039',
    # '0032_ANON7649583',
    # '0033_ANON9728185',
    # '0035_ANON3474225',
    # '0036_ANON0282755',
    # '0037_ANON0369080',
    # '0039_ANON0604912',
    # '0042_ANON9423619',
    # '0043_ANON7041133',
    # '0044_ANON8232550',
    # '0045_ANON2563804',
    # '0047_ANON3613611',
    # '0048_ANON6365688',
    # '0049_ANON9783006',
    # '0051_ANON1327674',
    # '0052_ANON9710044',
    # '0053_ANON5517301',
    # '0055_ANON3357872',
    # '0056_ANON2124757',
    # '0057_ANON1070291',
    # '0058_ANON9719981',
    # '0059_ANON7955208',
    # '0061_ANON7642254',
    # '0062_ANON0319974',
    # '0063_ANON9972960',
    # '0064_ANON0282398',
    # '0067_ANON0913099',
    # '0068_ANON7978458',
    # '0069_ANON9840567',
    # '0070_ANON5223499',
    # '0071_ANON9806291',
    # '0073_ANON5954143',
    # '0075_ANON5895496',
    # '0076_ANON3983890',
    # '0077_ANON8634437',
    # '0078_ANON6883869',
    # '0079_ANON8828023',
    # '0080_ANON4499321',
    # '0081_ANON9763928',
    # '0082_ANON6073234',
    # '0083_ANON9898497',
    # '0084_ANON6141178',
    # '0085_ANON4535412',
    # '0086_ANON8511628',
    # '0087_ANON9534873',
    # '0088_ANON9892116',
    # '0089_ANON9786899',
    # '0090_ANON0891692',
    # '0092_ANON9941969',
    # '0093_ANON9728761',
    # '0094_ANON8024204',
    # '0095_ANON4189062',
    # '0097_ANON5642073',
    # '0103_ANON8583296',
    # '0104_ANON7748630',
    # '0105_ANON9883201',
    # '0107_ANON4035085',
    # '0108_ANON0424679',
    # '0109_ANON9816976',
    # '0110_ANON8266491',
    # '0111_ANON9310466',
    # '0112_ANON3210850',
    # '0113_ANON9665113',
    # '0115_ANON0400743',
    # '0116_ANON9223478',
    # '0118_ANON7141024',
    # '0119_ANON3865800',
    # '0120_ANON7275574',
    # '0121_ANON9629161',
    # '0123_ANON7265874',
    # '0124_ANON8610762',
    # '0125_ANON0272089',
    # '0126_ANON4747182',
    # '0127_ANON8023509',
    # '0128_ANON8627051',
    # '0129_ANON5344332',
    # '0135_ANON9879440',
    # '0136_ANON8096961',
    # '0137_ANON8035619',
    # '0138_ANON1747790',
    # '0139_ANON2666319',
    # '0140_ANON0899488',
    # '0141_ANON8018038',
    # '0142_ANON7090827',
    # '0143_ANON9752849',
    # '0144_ANON2255419',
    # '0145_ANON0335209',
    # '0146_ANON7414571',
    # '0148_ANON9604223',
    # '0149_ANON4712664',
    # '0150_ANON5824292',
    # '0152_ANON2411221',
    # '0153_ANON5958718',
    # '0155_ANON7828652',
    # '0157_ANON9873056',
    # '0159_ANON9720717',
    # '0160_ANON3504149'
]

roots = {
    'kspace':            Path('/scratch/p290820/datasets/003_umcg_pst_ksps'),
    'reader_study':      Path('/scratch/hb-pca-rad/projects/03_reader_set_v2'),
    'reader_study_segs': Path('/scratch/hb-pca-rad/projects/03_reader_set_v2/segs'),
    'R3_gaussian':       Path(f"/scratch/hb-pca-rad/projects/04_uncertainty_quantification/gaussian/recons_{3}x"),
    'R6_gaussian':       Path(f"/scratch/hb-pca-rad/projects/04_uncertainty_quantification/gaussian/recons_{6}x"),
    'db_fpath_old':      Path('/scratch/p290820/datasets/003_umcg_pst_ksps/database/dbs/master_habrok_20231106_v2.db'),                  # References an OLDER version of the databases where the info could also just be fine that we are looking for
    'db_fpath_new':      Path('/home1/p290820/repos/Uncertainty-Quantification-Prostate-MRI/databases/master_habrok_20231106_v2.db'),    # References the LATEST version of the databases where the info could also just be fine that we are looking for
    'figures_gaussian':  Path('/scratch/hb-pca-rad/projects/04_uncertainty_quantification/gaussian/figures'),
}

acc_factors         = [3, 6] # Define the set of acceleration factors we care about.
acc_factors         = [3] # Define the set of acceleration factors we care about.

tablename = 'echo_train_mapping_debug'
tablename = 'echo_train_mapping'

In [ ]:
def load_kspace_data(pat_ids: List[str], acc_factors: List[int], roots: Dict[str, Path]) -> List[Dict]:
    """
    Load k-space data and ISMRMRD headers from h5 files.

    This function reads the header data as a string while the file is open,
    ensuring the stored header is valid after file closure.

    Parameters:
        pat_ids (list): List of patient IDs.
        acc_factors (list): List of acceleration factors.
        roots (dict): Dictionary with key 'kspace' pointing to the root directory.

    Returns:
        list: A list of dictionaries, each containing patient ID, acceleration factor,
              k-space data, and the header string.
    """
    kspaces = []
    for patient_id in pat_ids:
        for acc in acc_factors:
            kspace_dir = roots['kspace'] / 'data' / patient_id / 'h5s'
            kspace_files = list(kspace_dir.glob("*.h5"))
            assert len(kspace_files) == 1, f"Expected 1 kspace file for {patient_id}, found {len(kspace_files)}"
            
            with h5py.File(kspace_files[0], 'r') as f:
                kspace_data = f['kspace'][:]
                
                # Convert the header dataset to a string while the file is open.
                header_dataset = f['ismrmrd_header']
                header_str = header_dataset.asstr()[()]
                kspaces.append({
                    'id': patient_id,
                    'acc': acc,
                    'kspace': kspace_data,
                    'ismrmrd_header': header_str,
                })
            print(f"Loaded kspace data for patient {patient_id} with acceleration factor {acc}")
            
    return kspaces


kspaces = load_kspace_data(pat_ids, acc_factors, roots)

In [ ]:
def print_ismrmrd_headers(kspaces):
    """
    Print the ISMRMRD headers from each k-space entry.
    
    Parameters:
        kspaces (list): List of dictionaries containing header strings.
    """
    for entry in kspaces:
        print(f"Patient ID: {entry['id']}, Acc Factor: {entry['acc']}")
        print("ISMRMRD Header:")
        print(entry['ismrmrd_header'])
        print("-" * 80)

print_ismrmrd_headers(kspaces)

In [ ]:
# Here is the code to load kspace data and print the ISMRMRD headers.

# 1. Kspace Loading
avg1, avg2, avg3 = 0, 1, 2
kspaces = [] # Placeholder for kspace data list of dicts with Dict[str, Any]
for id in pat_ids:
    for acc in acc_factors:
        kspace_fpath = roots['kspace'] / 'data' / id / 'h5s' 
        kspace_fpaths = list(kspace_fpath.glob(f"*.h5"))
        assert len(kspace_fpaths) == 1, f"Expected 1 kspace file for {id}, found {len(kspace_fpaths)}"
        with h5py.File(kspace_fpaths[0], 'r') as f:
            print(f"\nLoaded data for {id} with acceleration factor {acc}")
            kspaces.append(
                {
                    'id': id,
                    'acc': acc,
                    'kspace':  f['kspace'][:],
                    'ismrmrd_header': f['ismrmrd_header'],
                }               
            )
            # print all key value pairs of the h5 file and the attributes
            for k, v in f.items():
                print(f"{k}: {v}")
            for k in f.attrs.keys():
                print(f"{k} => {f.attrs[k]}")

        print(f"Breaking after {id} with acceleration factor {acc}")
        break


# 2.Printing
for pat_dict in kspaces:
    print(f"Patient ID: {pat_dict['id']}")
    print(f"Acceleration Factor: {pat_dict['acc']}")
    print(f"Kspace shape: {pat_dict['kspace'].shape}")
    print(f"Kspace data type: {pat_dict['kspace'].dtype}")
    print(f"ISMRMRD header: {pat_dict['ismrmrd_header']}")
    print("\n")